In [1]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
# ##### 各関数での実行時間を保存するためのCSVの空データを作成するための処理 #####
# Processes = []
# for i in range(1,16):
#     Processes.append(i*i)
# for j in range(9):
#     Processes.append(2**j)
# Processes = list(set(Processes))
# Processes.sort()
# index = classes
# EmptyDF = pd.DataFrame(index=index, columns=Processes)
# for benchmark in benchmarks:
#     EmptyDF.to_csv(f"./csv_files/ExecTime@{benchmark}.csv")

##### 各ベンチマークの実行時間を記録する処理
# # Benchmarks = []
# # for benchmark in benchmarks:
# #     Benchmarks.append(benchmark.upper())
# # print(Benchmarks)

# Processes = [8, 16, 32, 64, 128, 256]
# for process in Processes:
#     TimeDataList = return_TimeDataList(process)
#     for DictData in TimeDataList:
#         benchmark = DictData.benchmark
#         process = DictData.process
#         BenchmarkClass = DictData.BenchmarkClass
#         time = DictData.time
#         FillCSV(benchmark=benchmark, process=process, BenchmarkClass=BenchmarkClass, time=time)

# # print(TimeDataList[0].return_AllData())


In [2]:
# 関数：BestModelsInDF()
# 引数：return_FixedClassModelDF()の返値
# 返値：引数で渡されたDFの要素のモデル名を要素としたDF
def BestModelsInDF(inputDF):
    returnDF = pd.DataFrame()
    returnDF = returnDF.reindex_like(inputDF).astype('str')
    index = returnDF.index.tolist()
    columns = returnDF.columns.tolist()
    for i in index:
        for j in columns:
            returnDF.at[i, j] = inputDF.at[i, j].ModelName()
    return(returnDF)
# ### 使用例
# benchmark = "mg"
# FixClass = "B"
# sampleDF = return_FixedClassModelDF(benchmark = benchmark, FixClass = FixClass)
# print(BestModelsInDF(inputDF=sampleDF))

# for benchmark in ['bt', 'mg', 'sp']:
FixClass = "B"
for benchmark in ['mg']:
    print(f"benchmark={benchmark}")
    TargetDF = return_FixedClassModelDF(benchmark=benchmark, FixClass=FixClass)
    BestModelsInTargetDF = BestModelsInDF(TargetDF)
    print(BestModelsInTargetDF)
    TargetDF = return_ErrorRateFixedClassDFwithAverage(benchmark = benchmark , FixClass=FixClass)
    print(TargetDF)

benchmark=mg
                 [1, 2, 4, 8, 16, 32, 64, 128, 256]  \
.TAU_application                           ModelLin   
ALLOC_SPACE                                ModelLin   
BUBBLE                                   ModelLog10   
COMM3                                      ModelLin   
COMM3_EX                                   ModelLin   
INTERP                                     ModelLin   
MG3P                                       ModelLin   
MG_MPI                                     ModelLin   
MPI_Allreduce()                            ModelLin   
MPI_Barrier()                              ModelLin   
MPI_Bcast()                                ModelLin   
MPI_Comm_rank()                            ModelLin   
MPI_Comm_size()                            ModelLin   
MPI_Finalize()                             ModelLin   
MPI_Init()                                 ModelLin   
MPI_Reduce()                               ModelLin   
NORM2U3                                    ModelLin 

In [3]:
# for benchmark in ['bt', 'mg', 'sp']:
for benchmark in ['mg']:
    ProcessExcludeBTSP = [1,2,4,8,16,32,64,128,256]
    ProcessIncludeBTSP = [1, 4, 16, 64, 256]

    TestRates = [0, 0.2, 0.4, 0.6, 0.8]
    if(benchmark == "bt" or benchmark == "sp"):
        processes=ProcessIncludeBTSP
    else:
        processes=ProcessExcludeBTSP
    RawDataList = return_fixed_class(BenchMark=benchmark, Processes=processes,FixedBenchMarkClass=FixClass)
    RawDataDF = pd.concat(RawDataList, axis=1)
    print(RawDataDF)
print('SET_CLASS' in RawDataDF.index.tolist())
print(RawDataDF.loc['SET_CLASS'])

                     1       2       4        8       16        32        64   \
.TAU_application     1.0     1.0     1.0     1.00     1.0     1.000     1.000   
ALLOC_SPACE          1.0     1.0     1.0     1.00     1.0     1.000     1.000   
BUBBLE             608.0   575.0   542.0   525.25   499.5   468.125   444.219   
COMM1P            1461.0     NaN     NaN      NaN     NaN       NaN       NaN   
COMM1P_EX          441.0     NaN     NaN      NaN     NaN       NaN       NaN   
COMM3              487.0   487.0   487.0   487.00   487.0   487.000   487.000   
COMM3_EX           147.0   147.0   147.0   147.00   147.0   147.000   147.000   
INTERP             147.0   147.0   147.0   147.00   147.0   147.000   147.000   
MG3P                21.0    21.0    21.0    21.00    21.0    21.000    21.000   
MG_MPI               1.0     1.0     1.0     1.00     1.0     1.000     1.000   
MPI_Allreduce()     88.0    88.0    88.0    88.00    88.0    88.000    88.000   
MPI_Barrier()        6.0    

KeyError: 'SET_CLASS'

In [ ]:
for benchmark in benchmarks:
    print(return_ErrorRateFixedProcess_AveragePredictCostRealCost(benchmark=benchmark, FixProcess=64, predict_class="D"))
    print(return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark=benchmark, FixClass="B", predict_process=256))

"SET_CLASS"に代表される関数が、プロファイル数が2個のときだけ突飛な平均誤差率を出力する。その原因は学主データ二個だと基本的に線形モデルを採用することになるが、そうすると、実際の最適モデルが反比例や対数だった場合に予測値と実測値が大幅にずれる。したがって、プロファイル数が2個のときにだけ突飛な平均誤差率を出力する。

実際に予測することになる関数のみだけを、表示するようにして、出力されるデータを見やすくする(↑の話)。

表のタイトルをそのまま保存時のファイル名にすればええんか